Copyright © 2023, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# HMEQ Dataset : Build and Import Trained Binary Classification Models into SAS Model Manager

This notebook provides an example of how to build and train a simple Python model and then import the model into SAS Model Manager (on either SAS Viya 3.5 or SAS Viya 4) using the HMEQ data set. Lines of code that must be modified by the user, such as directory paths or the host server are noted with the comment "_Changes required by user._".

_**Note:** If you download only this notebook and not the rest of the repository, you must also download the hmeq.csv file from the data folder in the examples directory. These files are used when executing this notebook example._

Here are the steps shown in this notebook:

1. Import, review, and preprocess data for model training.
2. Build, train, and assess a scikit-learn decision tree, random forest, and gradient boosting model.
3. Serialize the models into separate pickle files.
4. Write the metadata JSON files needed for importing into SAS Model Manager as well as optional files for fit statistics and ROC/Lift charts.
4. Write a score code Python file for model scoring.
5. Zip the pickle, JSON, and score code files into an archive file.
6. Import the ZIP archive file to SAS Model Manager via the Session object and relevant function call.

### Python Package Imports

In [ ]:
# Standard Library
from pathlib import Path
import warnings

# Third Party
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Application Specific
import sasctl.pzmm as pzmm
from sasctl import Session

In [ ]:
# Global Package Options
pd.options.mode.chained_assignment = None  # default="warn"
plt.rc("font", size=14)
# Ignore warnings from pandas about SWAT using a feature that will be depreciated soon
warnings.simplefilter(action="ignore", category=FutureWarning)

### Import and Review Data Set

In [ ]:
hmeq_data = pd.read_csv("data/hmeq.csv", sep= ",")
hmeq_data.shape

In [ ]:
hmeq_data.head()

In [ ]:
hmeq_data.hist(figsize=(15,15), layout=(4, 4));

In [ ]:
hmeq_data.columns

### Preprocess Data

In [ ]:
predictor_columns = ["LOAN", "MORTDUE", "VALUE", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]

target_column = "BAD"
x = hmeq_data[predictor_columns]
y = hmeq_data[target_column]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
# For missing values, impute the data set's mean value
x_test.fillna(x_test.mean(), inplace=True)
x_train.fillna(x_train.mean(), inplace=True)

### Create, Train, and Assess Model

In [ ]:
dtc = DecisionTreeClassifier(max_depth=7, min_samples_split=2, min_samples_leaf=2, max_leaf_nodes=500)
dtc = dtc.fit(x_train, y_train)

rfc = RandomForestClassifier(n_estimators=120, max_depth=9, max_leaf_nodes=850, min_samples_split=5, min_samples_leaf=4)
rfc = rfc.fit(x_train, y_train)

gbc = GradientBoostingClassifier(learning_rate=0.1, n_estimators=600, max_depth=4, min_samples_leaf=1, min_samples_split=2, max_leaf_nodes=500)
gbc = gbc.fit(x_train, y_train)

In [ ]:
# Calculate the importance of a predictor 
def sort_feature_importance(model, data):
    features = {}
    for importance, name in sorted(zip(model.feature_importances_, data.columns), reverse=True):
        features[name] = str(np.round(importance*100, 2)) + "%"
    return features

In [ ]:
importances = pd.DataFrame.from_dict(sort_feature_importance(dtc, x_train), orient="index").rename(columns={0: "DecisionTree"})
importances["RandomForest"] = pd.DataFrame.from_dict(sort_feature_importance(rfc, x_train), orient="index")
importances["GradientBoosting"] = pd.DataFrame.from_dict(sort_feature_importance(gbc, x_train), orient="index")
importances

In [ ]:
y_dtc_predict = dtc.predict(x_test)
y_dtc_proba = dtc.predict_proba(x_test)
print(confusion_matrix(y_test, y_dtc_predict))
print(classification_report(y_test, y_dtc_predict))
print("Decision Tree Model Accuracy = " + str(np.round(dtc.score(x_test, y_test)*100,2)) + "%")

In [ ]:
y_rfc_predict = rfc.predict(x_test)
y_rfc_proba = rfc.predict_proba(x_test)
print(confusion_matrix(y_test, y_rfc_predict))
print(classification_report(y_test, y_rfc_predict))
print("Random Forest Model Accuracy = " + str(np.round(rfc.score(x_test, y_test)*100,2)) + "%")

In [ ]:
y_gbc_predict = gbc.predict(x_test)
y_gbc_proba = gbc.predict_proba(x_test)
print(confusion_matrix(y_test, y_gbc_predict))
print(classification_report(y_test, y_gbc_predict))
print("Gradient Boosting Model Accuracy = " + str(np.round(gbc.score(x_test, y_test)*100,2)) + "%")

### Register Model in SAS Model Manager with pzmm

In [ ]:
# Model names within SAS Model Manager
model_prefix = ["DecisionTreeClassifier", "RandomForest", "GradientBoosting"]
# Directory location for the model files
zip_folder = [Path.cwd() / "data/hmeqModels/DecisionTreeClassifier/",
             Path.cwd() / "data/hmeqModels/RandomForest/",
             Path.cwd() / "data/hmeqModels/GradientBoosting"] # Changes required by user
model = [dtc, rfc, gbc]
# Output variables expected in SAS Model Manager. If a classification value is expected to be output, it should be the first metric.
score_metrics = ["EM_CLASSIFICATION", "EM_EVENTPROBABILITY"]

# Serialize the models to a pickle format
for (mod, prefix, path) in zip(model, model_prefix, zip_folder):
    pzmm.PickleModel.pickle_trained_model(
        model_prefix=prefix,
        trained_model=mod,
        pickle_path=path
    )

In [ ]:
def write_json_files(data, predict, target, path, prefix):    
    # Write input variable mapping to a json file
    pzmm.JSONFiles.write_var_json(input_data=data[predict], is_input=True, json_path=path)
    
    # Set output variables and assign an event threshold, then write output variable mapping
    output_var = pd.DataFrame(columns=score_metrics, data=[["A", 0.5]]) # data argument includes example expected types for outputs
    pzmm.JSONFiles.write_var_json(output_var, is_input=False, json_path=path)
    
    # Write model properties to a json file
    pzmm.JSONFiles.write_model_properties_json(
        model_name=prefix, 
        target_variable=target, # Target variable to make predictions about (BAD in this case)
        target_values=["1", "0"], # Possible values for the target variable (1 or 0 for binary classification of BAD)
        json_path=path, 
        model_desc=f"Description for the {prefix} model.",
        model_algorithm="",
        modeler="sasdemo",
    )
    
    # Write model metadata to a json file so that SAS Model Manager can properly identify all model files
    pzmm.JSONFiles.write_file_metadata_json(model_prefix=prefix, json_path=path)

for (prefix, path) in zip(model_prefix, zip_folder):
    write_json_files(hmeq_data, predictor_columns, target_column, path, prefix)

In [ ]:
import getpass
def write_model_stats(x_train, y_train, test_predict, test_proba, y_test, model, path, prefix):
    # Calculate train predictions
    train_predict = model.predict(x_train)
    train_proba = model.predict_proba(x_train)
    
    # Assign data to lists of actual and predicted values
    train_data = pd.concat([y_train.reset_index(drop=True), pd.Series(train_predict), pd.Series(data=train_proba[:,1])], axis=1)
    test_data = pd.concat([y_test.reset_index(drop=True), pd.Series(test_predict), pd.Series(data=test_proba[:,1])], axis=1)
    
    # Calculate the model statistics, ROC chart, and Lift chart; then write to json files
    pzmm.JSONFiles.calculate_model_statistics(
        target_value=1, 
        prob_value=0.5, 
        train_data=train_data, 
        test_data=test_data, 
        json_path=path
    )

    full_training_data = pd.concat([y_train.reset_index(drop=True), x_train.reset_index(drop=True)], axis=1)

    pzmm.JSONFiles.generate_model_card(
        model_prefix=prefix,
        model_files = path,
        algorithm = str(type(model).__name__),
        train_data = full_training_data,
        train_predictions=train_predict,
        target_type='classification',
        target_value=1,
        interval_vars=predictor_columns,
        selection_statistic='_RASE_',
    )
        
username = getpass.getpass()
password = getpass.getpass()
host = "demo.sas.com"# Changes required by user
sess = Session(host, username, password, protocol="http") # For TLS-enabled servers, change protocol value to "https"
conn = sess.as_swat() # Connect to SWAT through the sasctl authenticated connection

test_predict = [y_dtc_predict, y_rfc_predict, y_gbc_predict]
test_proba = [y_dtc_proba, y_rfc_proba, y_gbc_proba]
for (mod, pred, proba, path, prefix) in zip(model, test_predict, test_proba, zip_folder, model_prefix):
    write_model_stats(x_train, y_train, pred, proba, y_test, mod, path, prefix)

In [ ]:
for (prefix, path) in zip(model_prefix, zip_folder):
    pzmm.ImportModel.import_model(
        model_files=path, # Where are the model files?
        model_prefix=prefix, # What is the model name?
        project="HMEQModels", # What is the project name?
        input_data=x, # What does example input data look like?
        predict_method=[dtc.predict_proba, [int, int]], # What is the predict method and what does it return?
        score_metrics=score_metrics, # What are the output variables?
        overwrite_model=True, # Overwrite the model if it already exists?
        target_values=["0", "1"], # What are the expected values of the target variable?
        target_index=1, # What is the index of the target value in target_values?
        model_file_name=prefix + ".pickle", # How was the model file serialized?
        missing_values=True # Does the data include missing values?
    )
    # Reinitialize the score_code variable when writing more than one model's score code
    pzmm.ScoreCode.score_code = ""

### Implementing Score Testing

In [ ]:
import requests
import sys

from sasctl._services.score_definitions import ScoreDefinitions as sd # Importing ScoreDefinitions service
from sasctl._services.score_execution import ScoreExecution as se # Importing ScoreExecution service

In [ ]:
# Creating the score definition for this model using the model UUID generated two steps before
score_definition = sd.create_score_definition(
    "example_score_def_name", # Name of the score_definition, which can be any string
    "model_id",  # Use Model UUID generated two steps before
    "table_name", # Table name for input data, which must exist in host server or it will throw an HTTP error and prompt you to upload a data file
    # True,  # Uncomment 'True' if your Viya version is compatible with CAS Gateway
)

In [ ]:
# Executing the score definition
score_execution = se.create_score_execution(
    score_definition.get("id") # Score definition id created in the previous cell
)

# Prints score_execution_id
print(score_execution)

We can view our scored model information within Model Manager under Projects -> Choose your model -> Scoring. 
***